In [1]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from secrets import *

In [2]:
league_id = 84057

In [ ]:
# add https://stmorse.github.io/journal/espn-fantasy-3-python.html and other related links to readme

### Types of data to explore
1. kona player data
    * player stats
    * draft auction value
    * draft rankings

In [ ]:
# need league team info, so I can relate id to teams...I could also just pull these from team web pages
# anywhere in the API do they have game log stats?

### Kona player data for 2019

In [4]:
url = 'https://fantasy.espn.com/apis/v3/games/FBA/seasons/2019/segments/0/leagues/' + str(league_id)
params={"view": "kona_player_info"}
cookies = {"swid": AUTH['swid'],
           "espn_s2": AUTH['espn_s2']}
r = requests.get(url, params=params, cookies=cookies)
data = r.json()
data.keys()

dict_keys(['players'])

In [4]:
data['players'][100].keys()

dict_keys(['draftAuctionValue', 'id', 'keeperValue', 'keeperValueFuture', 'lineupLocked', 'onTeamId', 'player', 'ratings', 'rosterLocked', 'status', 'tradeLocked'])

In [5]:
data['players'][100]['draftAuctionValue']

0

In [12]:
data['players'][100]['onTeamId']

6

In [13]:
data['players'][100]['ratings']

{'0': {'positionalRanking': 15, 'totalRanking': 100, 'totalRating': 4.9421916},
 '1': {'positionalRanking': 28, 'totalRanking': 151, 'totalRating': 1.5701568},
 '2': {'positionalRanking': 29, 'totalRanking': 162, 'totalRating': 1.4011277},
 '3': {'positionalRanking': 17, 'totalRanking': 116, 'totalRating': 3.1654348}}

In [17]:
data['players'][100]['player'].keys()

dict_keys(['active', 'defaultPositionId', 'draftRanksByRankType', 'droppable', 'eligibleSlots', 'firstName', 'fullName', 'id', 'injured', 'injuryStatus', 'jersey', 'lastName', 'lastNewsDate', 'lastVideoDate', 'ownership', 'proTeamId', 'seasonOutlook', 'stats'])

In [19]:
data['players'][100]['player']['fullName']

'P.J. Tucker'

In [26]:
data['players'][100]['player']['injured']

False

In [20]:
data['players'][100]['player']['draftRanksByRankType']

{'STANDARD': {'auctionValue': 0,
  'published': False,
  'rank': 163,
  'rankSourceId': 0,
  'rankType': 'STANDARD',
  'slotId': 0}}

In [21]:
data['players'][100]['player']['ownership']

{'activityLevel': 'ACTIVE_LAST_DAY',
 'auctionValueAverage': 0.010101010101010102,
 'auctionValueAverageChange': 0.0,
 'averageDraftPosition': 139.53136094674556,
 'averageDraftPositionPercentChange': 0.0,
 'date': 1560431904264,
 'leagueType': 0,
 'percentChange': 0.000751755907183238,
 'percentOwned': 6.405699958634779,
 'percentStarted': 4.020868331968562}

### Stats

In [15]:
stat_list = []
team_id_list = []
for player in data['players']:
    #print(player['player']['fullName'])
    if 'stats' in player['player'].keys():
        for stat_item in player['player']['stats']:
            #looks like season totals AND averages id = '002019'
            if stat_item['id'] == '002019':
                if '40' in stat_item['stats'].keys(): # '40' is minutes
                    stat_item['stats']['playerName'] = player['player']['fullName']
                    stat_list.append(stat_item['stats'])
                    team_id_list.append(player['onTeamId'])

In [21]:
stats_df = pd.DataFrame(stat_list)
stats_df['onTeamId'] = team_id_list
stats_df.shape

(535, 46)

In [22]:
col_rename_dict = {'0':'pts', '1':'blocks', '2':'steals', '3':'ast', '6':'reb', '13':'fgm', '14': 'fga', 
                   '15': 'ftm', '16':'fta', '17':'threes', '40':'min', '42':'gp'}
stats_df.rename(columns=col_rename_dict, inplace=True)
stats_df['fg_pct'] = stats_df['fgm'] / stats_df['fga']
stats_df['ft_pct'] = stats_df['ftm'] / stats_df['fta']
stats_df = stats_df[['playerName','onTeamId','pts','blocks','steals','ast','reb','fgm','fga','fg_pct','ftm','fta','ft_pct','threes','min','gp']]

In [23]:
stats_df.head()

,playerName,onTeamId,pts,blocks,steals,ast,reb,fgm,fga,fg_pct,ftm,fta,ft_pct,threes,min,gp
0,Vince Carter,0,562.0,27.0,44.0,87.0,194.0,196.0,468.0,0.418803,47.0,66.0,0.712121,123.0,1327.0,76.0
1,Jamal Crawford,0,508.0,12.0,33.0,229.0,85.0,174.0,438.0,0.397260,93.0,110.0,0.845455,67.0,1210.0,64.0
2,Dirk Nowitzki,0,373.0,18.0,9.0,35.0,158.0,135.0,376.0,0.359043,39.0,50.0,0.780000,64.0,794.0,51.0
3,Tyson Chandler,0,173.0,23.0,21.0,37.0,307.0,61.0,99.0,0.616162,51.0,87.0,0.586207,0.0,876.0,55.0
4,Pau Gasol,0,118.0,15.0,5.0,52.0,137.0,42.0,94.0,0.446809,28.0,40.0,0.700000,6.0,360.0,30.0


In [24]:
stats_df.loc[stats_df.onTeamId==10]

,playerName,onTeamId,pts,blocks,steals,ast,reb,fgm,fga,fg_pct,ftm,fta,ft_pct,threes,min,gp
33,LaMarcus Aldridge,10,1727.0,107.0,43.0,194.0,744.0,684.0,1319.0,0.518575,349.0,412.0,0.847087,10.0,2690.0,81.0
47,Kevin Durant,10,2027.0,84.0,58.0,457.0,497.0,721.0,1383.0,0.521330,448.0,506.0,0.885375,137.0,2703.0,78.0
69,Robin Lopez,10,704.0,78.0,11.0,89.0,286.0,304.0,535.0,0.568224,89.0,123.0,0.723577,7.0,1609.0,74.0
132,Enes Kanter,10,915.0,26.0,32.0,116.0,659.0,375.0,683.0,0.549048,155.0,197.0,0.786802,10.0,1637.0,67.0
156,Jae Crowder,10,951.0,31.0,64.0,133.0,384.0,318.0,797.0,0.398996,142.0,197.0,0.720812,173.0,2168.0,80.0
158,Andre Drummond,10,1370.0,138.0,136.0,112.0,1232.0,561.0,1052.0,0.533270,243.0,412.0,0.589806,5.0,2646.0,79.0
168,Damian Lillard,10,2067.0,34.0,88.0,551.0,371.0,681.0,1533.0,0.444227,468.0,513.0,0.912281,237.0,2837.0,80.0
187,Seth Curry,10,581.0,12.0,36.0,66.0,120.0,212.0,465.0,0.455914,44.0,52.0,0.846154,113.0,1402.0,74.0
245,Larry Nance Jr.,10,627.0,40.0,100.0,214.0,552.0,249.0,479.0,0.519833,96.0,134.0,0.716418,33.0,1794.0,67.0
248,Dewayne Dedmon,10,693.0,71.0,69.0,90.0,480.0,259.0,526.0,0.492395,92.0,113.0,0.814159,83.0,1608.0,64.0
